# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [4]:
#Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [3]:
#Your code here; create the connection and cursor
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Select the Average Number of Orders on a Product by Product Basis

In [7]:
#Your code here; use a subquery. No join will be necessary.
df = pd.read_sql_query("""SELECT AVG(count)
                            FROM (SELECT productCode, COUNT(*) as count
                                    FROM orderdetails
                                    GROUP BY 1)
                            ;""", conn)
df.head()

,AVG(count)
0,27.486239


## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [27]:
#Your code here; use a subquery. No join will be necessary.
df = pd.read_sql_query("""SELECT customerNumber, contactLastName, contactFirstName
                            FROM customers
                            WHERE customerNumber in (SELECT customerNumber
                                    FROM orders
                                    WHERE orderDate = '2003-01-31')
                            ;""", conn)
df.head()

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [87]:
#Your code here
df = pd.read_sql_query("""SELECT productCode, SUM(quantityOrdered)
                            FROM products
                            JOIN orderdetails
                            USING (productCode)
                            GROUP BY productCode
                            ORDER BY SUM(quantityOrdered) DESC;
                            """, conn)
df.head()

,productCode,SUM(quantityOrdered)
0,S18_3232,1808
1,S18_1342,1111
2,S700_4002,1085
3,S18_3856,1076
4,S50_1341,1074


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [93]:
#Your code here
df = pd.read_sql_query("""SELECT productName, Total_Num_Customers 
                            FROM products
                            JOIN
                            (SELECT productCode, COUNT(DISTINCT(customerNumber)) AS Total_Num_Customers
                                FROM orderdetails
                                JOIN orders
                                USING (orderNumber)
                                GROUP BY productCode)
                            USING(productCode)
                            ORDER BY Total_Num_Customers DESC
                            ;
                            """, conn)
df.head()

,productName,Total_Num_Customers,productCode
0,1992 Ferrari 360 Spider red,40,S18_3232
1,1952 Alpine Renault 1300,27,S10_1949
2,1972 Alfa Romeo GTA,27,S10_4757
3,1934 Ford V8 Coupe,27,S18_2957
4,Boeing X-32A JSF,27,S72_1253


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [103]:
#Your code here
df = pd.read_sql_query("""SELECT DISTINCT employeeNumber, officeCode, o.city, firstName, lastName
                            FROM employees 
                            JOIN offices o
                            USING (officeCode)
                            JOIN customers 
                            ON employeeNumber = salesRepEmployeeNumber
                            JOIN orders
                            USING (customerNumber)
                            JOIN orderdetails
                            USING (orderNumber)
                            WHERE productCode in (SELECT productCode
                                FROM orderdetails
                                JOIN orders
                                USING (orderNumber)
                                GROUP BY productCode
                                HAVING COUNT(DISTINCT(customerNumber)<20))
                            ;
                            """, conn)
df

,employeeNumber,officeCode,city,firstName,lastName
0,1216,2,Boston,Steve,Patterson
1,1504,7,London,Barry,Jones
2,1286,3,NYC,Foon Yue,Tseng
3,1370,4,Paris,Gerard,Hernandez
4,1401,4,Paris,Pamela,Castillo
5,1323,3,NYC,George,Vanauf
6,1621,5,Tokyo,Mami,Nishi
7,1501,7,London,Larry,Bott
8,1165,1,San Francisco,Leslie,Jennings
9,1337,4,Paris,Loui,Bondur


## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [110]:
#Your code here
df = pd.read_sql_query("""SELECT employeeNumber, firstName, lastName, COUNT(DISTINCT customerNumber)
                            FROM employees 
                            JOIN customers 
                            ON employeeNumber = salesRepEmployeeNumber
                            GROUP BY 1, 2, 3
                            HAVING AVG(creditLimit) > 15000
                            ;
                            """, conn)
df

,employeeNumber,firstName,lastName,COUNT(DISTINCT customerNumber)
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!